# Fiber Modes

If we assign U to any electric or magnetic field, the equation to solved is called the Helmholtz equation:

$$
\nabla^2 U + n^2(r)k_0^2 U=0
$$

These are the solutions:

$U(r,\phi,z)=u(r)e^{-jl\phi}e^{-j\beta z}$  for  $l=0,\pm 1,\pm 2...$

$$
u(r) = \left\{
\begin{array}{ll}
        J_l(k_T r) & r<a  \\
        K_l(\gamma r) & r>a
\end{array}
\right.
$$


where J and K are the Bessel functions of the first and second kind respectively.



# Imports

In [1]:
## Imports

import numpy as np
import scipy.special as spe
from scipy import optimize
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("bmh")

#import bqplot
#from bqplot import pyplot as bqplt

import ipywidgets as widgets
#from IPython.display import display

import time

pi = np.pi


## Initialization
dpi = plt.rcParams["figure.dpi"] 

mesh_limit = 2.
xx = np.linspace(-mesh_limit, mesh_limit, 60)
yy = np.linspace(-mesh_limit, mesh_limit, 60)

x_mesh, y_mesh = np.meshgrid(xx, yy)
r_mesh = np.sqrt(x_mesh ** 2 + y_mesh ** 2)
phi_mesh = np.arctan2(y_mesh, x_mesh)

ones_mesh = np.ones((len(xx), len(yy)))
zeros_mesh = np.zeros((len(xx), len(yy)))

in_core_mesh = ones_mesh.copy()
in_core_mesh[r_mesh > 1] = zeros_mesh[r_mesh > 1]  # mask core with ones

in_clad_mesh = ones_mesh.copy()
in_clad_mesh[r_mesh <= 1] = zeros_mesh[r_mesh <= 1]  # mask cladding with ones

# this is to plot the core perimeter later
phi_core_shape = np.linspace(0, 2 * pi, 60)
x_core_shape = 1 * np.cos(phi_core_shape)
y_core_shape = 1 * np.sin(phi_core_shape)

n_cladding = 1.444

## Functions

def zero_func(X, V, L):
    Y = np.sqrt(V ** 2 - X ** 2)
    return X * spe.jv(L + 1, X) / spe.jv(L, X) - Y * spe.kv(L + 1, Y) / spe.kv(L, Y)


def find_zeros_exact(X, Y, V, L):
    f = X * spe.jv(L + 1, X) / spe.jv(L, X) - Y * spe.kv(L + 1, Y) / spe.kv(L, Y)

    tt = len(X)
    zeros = []
    brackets = []

    for ii in range(tt - 1):
        
        if f[ii] * f[ii + 1] < 0:  # change of sign
            
            if ii != 0 and ii != tt - 2:  # not at an extreme
                
                if abs(f[ii - 1] - f[ii + 2]) > abs(f[ii] - f[ii + 1]):  # not an asymptote
                    brackets += [[X[ii], X[ii + 1]]]
                    
            else:
                brackets += [[X[ii], X[ii + 1]]]
    
    sols = []
    
    for br in brackets:
        optimum = optimize.root_scalar(zero_func, args=(V, L), bracket=br, method='brentq')
        sols.append(optimum)

    return [a.root for a in sols]


def calc_Er(mode,r):
    kt = mode.X / mode.a
    gamma = np.sqrt(mode.V ** 2 - mode.X ** 2) / mode.a
    Er = np.empty([len(r)])
    correction_clad = spe.jv(mode.L, kt * mode.a)/spe.kv(mode.L, gamma * mode.a)
    for ii in range(len(r)):
        if r[ii]<-mode.a:
            Er[ii] =spe.kv(mode.L, gamma * abs(r[ii]))*correction_clad
            if mode.L%2:
                Er[ii]=-Er[ii]
        elif r[ii]>mode.a:
            Er[ii] =spe.kv(mode.L, gamma * r[ii])*correction_clad
        else:
            Er[ii] = spe.jv(mode.L, kt * r[ii])
    
    Er = Er / max(abs(Er))
    return Er

def calc_E_mesh(mode,r_mesh,phi_mesh,in_core_mesh,in_clad_mesh):
    X = mode.X
    L = mode.L
    V = mode.V
    a = mode.a
    kt = X / a
    gamma = np.sqrt(V ** 2 - X ** 2) / a

    E_core = spe.jv(L, kt * a * r_mesh) * np.cos(L * phi_mesh)
    E_clad = spe.kv(L, gamma * a * r_mesh) * np.cos(L * phi_mesh) / spe.kv(L, gamma * a) * spe.jv(L, kt * a)
    E = E_core * in_core_mesh + E_clad * in_clad_mesh
    E = E/np.amax(E)
    return E

class Mode:
    pass


def find_modes(a=8.2 / 2, Na=0.12, n_cladding=1.444, w=1.55):
    """Calculates all the modes in the fiber, and puts them in the list of modes.
    It also returns the total number of modes, which is higher than len(modes) because some modes
    have degeneracy 2 (L=0) and some have degeneracy 4 (L>0)"""

    k0 = 2 * pi / w
    n_core = np.sqrt(Na ** 2 + n_cladding ** 2)

    V = k0 * a * Na
    phi = np.linspace(1E-10, pi / 2 - 1E-10, 5000)
    X = V * np.sin(phi)
    Y = V * np.cos(phi)

    solutions = True
    L = 0
    M = 1
    modes = []
    tot_modes = 0
    with np.errstate(invalid='ignore'):
        while solutions:
            sols = find_zeros_exact(X, Y, V, L)
        
            for sol in sols:
                mode = Mode()
                mode.X = sol
                mode.L = L
                mode.M = M
                mode.V = V
                mode.a = a

                mode.neff = np.sqrt(n_core**2 - (sol/(a*k0)) ** 2)
                mode.label = f"LP({L},{M})"

                mode.degeneracy = 2 if L == 0 else 4

                modes.append(mode)

                tot_modes += mode.degeneracy

                M += 1

            M = 1
            L += 1

            solutions = len(sols) != 0
            
    return modes, tot_modes


## Interactive elements

slider_diam = widgets.FloatSlider(min=0.1, max=80, value=8.2)
slider_Na = widgets.FloatSlider(min=0.0001, max=0.5, step=0.01, value=0.12)
slider_lambda = widgets.FloatSlider(min=0.5, max=2.0, step=0.01, value=1.55)
text_n_clad = widgets.FloatText(min = 1.0, max = 2.0, step = 0.01, value = 1.444)
label_n_core = widgets.Label()
label_V = widgets.Label()
label_V_max = widgets.Label(value = r'\(\color{red} {V max = 50!}\)')
label_V_max.layout.visibility = 'Hidden'
label_text_modes_found = widgets.Label()
btn_calc = widgets.Button(description='Calculate modes')
btn_calc.style.button_color = 'lightgray'
btn_plot = widgets.Button(description='Plot modes')
btn_plot.style.button_color = 'lightgray'
btn_plot.label_gen_plots = widgets.Label(value = 'Generating plots...')
btn_plot.label_gen_plots.layout.visibility = 'Hidden'

def update_text(obj):
    V = 2 * pi / slider_lambda.value * slider_Na.value * slider_diam.value / 2
    if V>50:
        label_V_max.layout.visibility = 'Visible'
        obj['owner'].value = obj['old']
        V = 2 * pi / slider_lambda.value * slider_Na.value * slider_diam.value / 2


    n_core = np.sqrt(slider_Na.value ** 2 + text_n_clad.value ** 2)
    label_n_core.value = f"{n_core:.4f}"
    label_V.value = f"{V:.4f}"
    return



update_text(0)

slider_Na.observe(update_text, names = 'value')    
slider_lambda.observe(update_text, names = 'value')
slider_diam.observe(update_text, names = 'value')
text_n_clad.observe(update_text, names = 'value')
text_n_clad.layout.width = '80px'
a = slider_diam.value / 2.0
Na = slider_Na.value
w = slider_lambda.value
V = 2 * pi / w * a * Na
n_clad = text_n_clad.value


# diam_core = 50
# Na=0.22 #typ. MM fiber

    


num_modes_show = 0

def btn_calc_eventhandler(obj):

    label_V_max.layout.visibility = 'hidden'

        
    label_text_modes_found.value = 'Calculating...'
    w = slider_lambda.value
    diam_core = slider_diam.value
    Na = slider_Na.value
    a = diam_core / 2
    r=np.linspace(-3*a,3*a,100)
    n_cladding = text_n_clad.value
    
    (modes, tot_modes) = find_modes(a=a, Na=Na, w=w, n_cladding=n_cladding)
    modes.sort(key=lambda x: x.neff, reverse=True)
    num_modes = len(modes)
    label_text_modes_found.value = f'Distinct modes found: {num_modes}. Total modes: {tot_modes}'
    obj.modes = modes
    obj.r = r
    return

 

def btn_plot_eventhandler(obj):
    obj.label_gen_plots.layout.visibility = 'Visible'
    for ii in range(len(obj.mode_row_list)):
        obj.mode_row_list[ii].children[1].clear_output(wait = True)
        obj.mode_fig_list[ii].clf()
    obj.mode_row_list = []
    obj.mode_fig_list = []
    
    modes = btn_calc.modes
    r=btn_calc.r
    #box_modes.children = []

    for ii in range(len(modes)):
        mode = modes[ii]
        
        show_text = f'<b>Mode {ii+1}</b>:<br>{mode.label}<br>Degeneracy {mode.degeneracy}<br>neff = {mode.neff:0.6f}'
        
        text_label = widgets.HTML(value=show_text)
        #text_label.layout.display = 'flex'
        text_label.layout.width = '150px'
        #text_label.layout.justify_content = 'flex-start'
        
        #mode_row_list[ii].children = [text_label, fig_rad_plot(modes[ii], r), fig_mode_profile(modes[ii])]
        mode_row = widgets.HBox([text_label,widgets.Output()],layout=mode_row_layout)
        #mode_row.children[1].layout.justify_content = 'flex-end'
        #mode_row.children[1].layout.border = 'solid 1px'
        obj.mode_row_list.append(mode_row)
        
        with mode_row.children[1]:
            fig, (ax1,ax2) = plt.subplots(1,2,figsize = (400/dpi,200/dpi))
            ax1.plot(r,calc_Er(modes[ii],r))
            
            ax1.set_xticks([-mode.a, mode.a])
            ax1.set_xticklabels(['-a','a'])
            ax1.set_xlim([r[0],r[-1]])
            ax1.set_ylabel('E(r)')
            ax1.set_yticks([0])
            ax1.set_yticklabels([])
            ax1.set_position([0.1,0.1,.4,.8])
            E=calc_E_mesh(mode,r_mesh,phi_mesh,in_core_mesh,in_clad_mesh)
            ax2.imshow(E,cmap='RdBu',vmin = -1., vmax = 1.,
                       extent = (xx[0],xx[-1],yy[0],yy[-1]),
                       aspect='equal')
            ax2.plot(x_core_shape,y_core_shape,'black',linewidth=0.5)
            ax2.set_xticks([])
            ax2.set_yticks([])
            ax2.set_position([0.6,0.1,.4,.8])
            
            plt.show(fig)
            obj.mode_fig_list.append(fig)
    obj.box_modes.children = obj.mode_row_list
    obj.label_gen_plots.layout.visibility = 'hidden'
    return
        
            

    


grid = widgets.GridspecLayout(5, 20)
#dict_justif_right = dict(display='flex',justify_items = 'flex-end')


grid[0, :4] = widgets.HTML('Core diameter (&mu;m)')
grid[1, 0:4] = widgets.HTML('Num. Aperture (NA)')
grid[2, :4] = widgets.HTML('Wavelength (&mu;m)')
grid[0, 4:10] = slider_diam
grid[1, 4:10] = slider_Na
grid[2, 4:10] = slider_lambda
grid[0, 14:16] = widgets.Label('n cladding')
grid[1,14:16] = widgets.Label('n core')
grid[1,16:17] = label_n_core
grid[2,14:16] = widgets.Label('V = 2$\pi$NAa/$\lambda$ =')
grid[2,16:17] = label_V
grid[2,17:19] = label_V_max
grid[0,16:17] = text_n_clad
grid[3, :4] = btn_calc
grid[3,4:12] = label_text_modes_found
grid[4,:4] = btn_plot
grid[4,5:12] = btn_plot.label_gen_plots



mode_row_layout = widgets.Layout(width='590px', height='200px', border='solid 1px', margin = '3px 3px 3px 3px',
                                justify_content = 'space-around')
#mode_row_layout = widgets.Layout(width='600px', height='150px')



btn_plot.mode_row_list = []


#for ii in range(3):
#    btn_plot.mode_row_list.append(widgets.Output(layout = mode_row_layout))
#    with btn_plot.mode_row_list[-1]:
#        fig, ax= plt.subplots(figsize = (2.8, 2))
#        btn_plot.mode_fig_list.append(fig)
#        plt.show(fig)

mode_list_layout = widgets.Layout(width='600px', height='', flex_flow='column', display='flex')
btn_plot.box_modes = widgets.Box(children=btn_plot.mode_row_list, layout=mode_list_layout)

display(grid)
display(btn_plot.box_modes)

btn_calc.modes = []
btn_calc.r=0.

btn_calc.on_click(btn_calc_eventhandler)
btn_plot.on_click(btn_plot_eventhandler)



GridspecLayout(children=(HTML(value='Core diameter (&mu;m)', layout=Layout(grid_area='widget001')), HTML(value…

Box(layout=Layout(display='flex', flex_flow='column', height='', width='600px'))